# SQLlite Queries From the Dataset of  a Digital Music Store

![title](dataschema.png)

### 1. Connect Dataset

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

- Return information on the tables and views in the database.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM invoice_line
LIMIT 10;

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1
6,1,1163,0.99,1
7,1,1164,0.99,1
8,1,1165,0.99,1
9,1,1166,0.99,1
10,1,1167,0.99,1


### 2. New Albums Purchase Recommendation

Makes a recommendation for the three artists whose albums we should purchase for the store, based on sales of tracks from their genres in USA.


- Returns each genre, with the number of tracks sold in USA: in absolute numbers, in percentages:

In [4]:
%%sql
WITH 
    sold_usa AS
    (
    SELECT 
        g.name genre_name,
        i.invoice_id invoice_id
     FROM genre g 
     INNER JOIN track t ON g.genre_id = t.genre_id
     INNER JOIN invoice_line il ON t.track_id = il.track_id
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
     WHERE c.country = 'USA'
    )

SELECT 
    genre_name,
    COUNT(invoice_id) genre_quantity,
        CAST(COUNT(invoice_id) AS FLOAT)/ 
        (SELECT COUNT(*) FROM sold_usa)
        genre_percent
FROM sold_usa 
GROUP BY 1
ORDER BY 2 DESC;
 

 * sqlite:///chinook.db
Done.


genre_name,genre_quantity,genre_percent
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Pop,22,0.02093244529019981
Latin,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


In the US, Rock, Punk and Metal are the top 3 music genre, accounting for almost 80% of the market share. All other genres accounted less than 1% each. 

Based on the sales performance, we should purchase the new albums by the following artists:

Red Tone (Punk)
Slim Jim Bites (Blues)
Meteor and the Girls (Pop)

We should also look out for artists and albumns from 'rock' genre which already accounts for more than half of the total sales in USA.

### 3. Emploees Sales Performance Analysis

Which sales support agent performed best?


- Return total dollar amount of sales assigned to each sales support agent within the company.

In [5]:
%%sql

WITH employee_customer AS
    (
    SELECT 
        e.*,
        ROUND(SUM(i.total),2) total_sales
    FROM employee e
    INNER JOIN customer c ON e.employee_id = c.support_rep_id
    INNER JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY e.employee_id
    ORDER BY total_sales DESC
    )
SELECT 
    first_name || ' ' || last_name employee_name,
    hire_date,
    total_sales
FROM employee_customer;



 * sqlite:///chinook.db
Done.


employee_name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


Jane Peacock had the best performance. The gap between the top sales person and the lowest person is about 20%, mainly due to the date of entry.

### 4. Sales Analysis by Country 
  
Analyze purchase from different countries. Countries which have only one customers are collected into "Other" group.


- Return total number of customers, total value of sales, average value of sales per customer, and average order value of each countries. 

In [6]:
%%sql

WITH
    customer_invoice_other AS
    (
    SELECT
        CASE WHEN(
                 SELECT COUNT(*)
                 FROM customer
                 WHERE country = c.country) 
             = 1 THEN 'Other'
             ELSE c.country
        END AS country,
        i.*
    FROM customer c 
    INNER JOIN invoice i ON c.customer_id = i.customer_id),
    
    country_value AS
    (
    SELECT
        country,
        count(distinct customer_id) total_customer,
        SUM(total) total_sales,
        CAST(SUM(total) AS FLOAT) / count(invoice_id) average_order_value,
        CAST(SUM(total) AS FLOAT) / count(distinct customer_id) average_customer_value
    FROM customer_invoice_other
    GROUP BY country
    ORDER BY total_customer DESC
    )

select 
    country,
    total_customer,
    total_sales,
    average_order_value,
    average_customer_value
FROM (
    SELECT 
        cv.*,
        CASE 
            WHEN country = 'Other' THEN 1
            ELSE 0
        END AS sort
    FROM country_value cv
    )
ORDER BY sort ASC; 
        

 * sqlite:///chinook.db
Done.


country,total_customer,total_sales,average_order_value,average_customer_value
USA,13,1040.4899999999998,7.942671755725189,80.0376923076923
Canada,8,535.5900000000001,7.047236842105265,66.94875000000002
France,5,389.0699999999999,7.781399999999998,77.81399999999998
Brazil,5,427.68000000000006,7.0111475409836075,85.53600000000002
Germany,4,334.62,8.161463414634147,83.655
United Kingdom,3,245.52,8.768571428571429,81.84
Portugal,2,185.13000000000002,6.383793103448276,92.56500000000001
India,2,183.14999999999998,8.72142857142857,91.57499999999999
Czech Republic,2,273.24000000000007,9.108000000000002,136.62000000000003
Other,15,1094.9400000000003,7.44857142857143,72.99600000000002



Based on the results, there may be opportunity in the following countries:

United Kingdom, 
Portugal, 
Czech Republic, 
India.

Because the amount of data from these countries are relatively low. But we should be careful on launching new marketing campaigns by spending too much money, as the sample data doest not provide enough information for the further step. 


### 5. Whether the music store should continue to buy full album from record company?
The music store is setup in a way that allows customer to make purchases in one of the two ways:  purchase a whole album, and purchase a collection of one or more individual tracks. In order to save money, they consider to change purchasing strategy - purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album. 

The store has previously done analysis to confirm that case, in which customers manually select every track from an album, and then add a few individual tracks from other albums to their purchase, does not happen often, so we can ignore this case.


- Return the percentage of purchases which are individual tracks vs whole albums to understand the effect this decision might have on overall revenue.
Identify whether each invoice has all the tracks from an album by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. Look up the album that one of the purchased tracks belongs to.

In [7]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) Number_of_invoices,
    CAST(COUNT(invoice_id) AS FLOAT) / 
    (SELECT COUNT(*) FROM invoice) Percentage_of_purchase
FROM
(
SELECT 
    ifs.*,
    CASE 
    WHEN
    (
    /*select invoice list*/
    SELECT 
        track_id
        FROM invoice_line il2 
        WHERE il2.invoice_id = ifs.invoice_id

     EXCEPT
     
     /*select album list*/
     SELECT 
         track_id
         FROM track t
         WHERE album_id=
         (SELECT t2.album_id
          FROM track t2 
          WHERE t2.track_id= ifs.first_track_id)
    ) IS NULL
    
    AND
    
    (
    SELECT 
         track_id
         FROM track t
         WHERE album_id=
         (SELECT t2.album_id
          FROM track t2 
          WHERE t2.track_id= ifs.first_track_id)
    EXCEPT

    select 
        track_id
        FROM invoice_line il2 
        WHERE il2.invoice_id = ifs.invoice_id
    ) IS NULL
    
    THEN 'Yes'
    ELSE 'No'
    END AS album_purchase
    FROM invoice_first_track ifs
)
GROUP BY 1;
    


 * sqlite:///chinook.db
Done.


album_purchase,Number_of_invoices,Percentage_of_purchase
No,500,0.8143322475570033
Yes,114,0.18566775244299674



Album purchases contribute 18.6% of purchases.  If only tracks were purchased from record company, the music store may risk losing a fifth of the revenue.